In [10]:
# imports

import h5py
import csv
import numpy as np
import os
import pandas as pd
import scipy


In [11]:
#Reading and combining the two csv files

# HDF5 file with exam_id and tracings
filename = "./data/exams_part0.hdf5"

# CSV file with exam_id, patient_Id and label
df_sample1 = pd.read_csv("./data/code15_chagas_labels.csv")

# main CSV file
df_sample2 = pd.read_csv("./data/exams.csv")

lista = ['exam_id','patient_id']

# csv file with exam_id, patient_id and label
df_master = pd.merge(df_sample2, df_sample1, on=lista, how='left')

# saving it as  a file
df_master.to_csv('data/exams_labels.csv', index=False)

In [12]:
#Saving important information in a dictionarys

# reading file HDF5 separing the two datasets
ecgs = h5py.File(filename, 'r')
exam_ids = np.array(ecgs['exam_id'])
tracings = np.array(ecgs['tracings'])

# pupulating the dictionary exam_ids_and_signals
exam_ids_to_chagas = dict()

with open("data/merged.csv", newline="\n") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        exam_id = int(row['exam_id'])
        boolean = bool(row['chagas'])
        exam_ids_to_chagas[exam_id] = boolean

# list of files to read
files = ["./data/exams_part"+str(i)+".hdf5" for i in range(0,18)]

# populating the dictionary exam_ids_and_signals
exam_ids_and_signals = dict()

for filename in files:
    # if the file doesn not exist, skip it
    if (os.path.exists(filename) == False):
        continue

    print("Reading file: ", filename)

    with h5py.File(filename, "r") as ecgs:

        exam_ids = list(ecgs['exam_id'])

        num_exams = len(exam_ids)

        done = False
        
        for i in range(num_exams):
            exam_id = exam_ids[i]
            
            if exam_id not in exam_ids_to_chagas:
                continue
            else:
                exam_ids_and_signals[exam_id] = ecgs['tracings'][i]
                if done == False:
                    print("Exam id: ", exam_id)
                    print("Signal: ", ecgs['tracings'][i])
                    done = True

# so at the end i got
# exam_ids_to_chagas that maps the exam id to Chagas disease label
# exam_ids_and_signals that maps the exam id to the ECG signal


Reading file:  ./data/exams_part0.hdf5
Exam id:  590673
Signal:  [[-4.8781034e-01 -2.6677129e-01  2.2103906e-01 ... -1.1737937e+00
  -5.5640870e-01 -4.8781034e-01]
 [-4.8106578e-01 -2.6019633e-01  2.2086945e-01 ... -1.1674997e+00
  -5.5151391e-01 -4.8232922e-01]
 [-4.7979322e-01 -2.5791711e-01  2.2187607e-01 ... -1.1599243e+00
  -5.4229915e-01 -4.7729760e-01]
 ...
 [-1.3724924e+00 -1.2811700e+00  9.1322273e-02 ... -8.7824506e-01
  -5.0396240e-01  8.5938675e-04]
 [-1.3667058e+00 -1.2758980e+00  9.0807825e-02 ... -8.6861408e-01
  -4.9709535e-01  1.6935735e-03]
 [-1.3614640e+00 -1.2694675e+00  9.1996796e-02 ... -8.5737538e-01
  -4.8235169e-01  1.0566236e-02]]
Reading file:  ./data/exams_part1.hdf5
Exam id:  4409337
Signal:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Reading file:  ./data/exams_part2.hdf5
Exam id:  18903
Signal:  [[ 0.2515272  -1.0137309  -1.2652582  ... -1.4024

In [13]:
# Bandpass filter

def butter_bandpass(lowcut, highcut, fs,order):

    nyquist_freq = 0.5 * fs
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    b,a = scipy.signal.butter(order, [low, high], btype='band')
    return b,a

def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    b,a = butter_bandpass(lowcut, highcut, fs, order)
    y = scipy.signal.lfilter(b, a, data)
    return y

# signal info for the bandpass filter
fs = 400
lowcut = 0.5
highcut = 40


filtered_signals = dict()
done = False
for exam_id, signal in exam_ids_and_signals.items():
    filtered_signal = butter_bandpass_filter(signal, lowcut, highcut, fs,3)
    filtered_signals[exam_id] = filtered_signal
    if done == False:
        print("Original signal: ", signal)
        print("Filtered signal: ", filtered_signal)
        done = True

print("Filtering done")
print(len(filtered_signals))

Original signal:  [[-4.8781034e-01 -2.6677129e-01  2.2103906e-01 ... -1.1737937e+00
  -5.5640870e-01 -4.8781034e-01]
 [-4.8106578e-01 -2.6019633e-01  2.2086945e-01 ... -1.1674997e+00
  -5.5151391e-01 -4.8232922e-01]
 [-4.7979322e-01 -2.5791711e-01  2.2187607e-01 ... -1.1599243e+00
  -5.4229915e-01 -4.7729760e-01]
 ...
 [-1.3724924e+00 -1.2811700e+00  9.1322273e-02 ... -8.7824506e-01
  -5.0396240e-01  8.5938675e-04]
 [-1.3667058e+00 -1.2758980e+00  9.0807825e-02 ... -8.6861408e-01
  -4.9709535e-01  1.6935735e-03]
 [-1.3614640e+00 -1.2694675e+00  9.1996796e-02 ... -8.5737538e-01
  -4.8235169e-01  1.0566236e-02]]
Filtered signal:  [[-0.00855057 -0.04540272 -0.10563203 ...  0.13635368  0.04497923
  -0.1603799 ]
 [-0.00843235 -0.04472437 -0.10387995 ...  0.14172893  0.05256662
  -0.15174786]
 [-0.00841004 -0.04457818 -0.10344445 ...  0.14306358  0.05527141
  -0.14735953]
 ...
 [-0.0240577  -0.13704448 -0.35080243 ... -0.07220544 -0.13121308
  -0.24428359]
 [-0.02395627 -0.13646895 -0.349336